# ICE3333 - Elementos Finitos No-Lineales 
## Material NeoHookean

En este archivo se presenta una alternativa para programar la función que nos permite obtener el tensor PK2 y el tensor elástico  ${\tt C}$ para un material NeoHookean. En particular, para este material sus componentes vienen dadas por

\begin{align*}
S_{IJ} &= \mu \delta_{IJ} + (\lambda \ln J - \mu) C^{-1}_{IJ},\\
C_{IJKL} &= \lambda C_{IJ}^{-1} C_{KL}^{-1}  + 2(\mu - \lambda \ln J) C_{IK}^{-1} C_{JL}^{-1},
\end{align*}





Para lo anterior programaremos la siguiente rutina:
$\texttt{[S, C] = material_NLElasticity_NH(material_state, material_parameters)}$ donde $\\\texttt{material_state}$ es el tensor Lagrangeano de deformación, ${\tt material \_parameters = [mu, lam]}$, ${\tt S}$ es el tensor PK2 y ${\tt C}$ es el tensor elástico material.

Dada la simetría de los tensores es conveniente utilizar notación de Voigt. Luego, de modo de obtener una mayor eficiencia, formaremos los tensores directamente en esta notación.


### Rutina material

In [1]:
import numpy as np



def material_NLElasticity_NH(material_state, material_parameters):
    mu,lam=material_parameters
    E=material_state
    Eten=np.matrix([[E[0,0],E[3,0]/2.,E[5,0]/2.],
                    [E[3,0]/2.,E[1,0],E[4,0]/2.],
                    [E[5,0]/2.,E[4,0]/2.,E[2,0]]])
    C=2.*Eten+np.eye(3)
    invC=np.linalg.inv(C)
    J=np.sqrt(np.linalg.det(C))
    Sten=mu*np.eye(3)+(lam*np.log(J)-mu)*invC
    S=np.matrix([[Sten[0,0],Sten[1,1],Sten[2,2],Sten[0,1],Sten[1,2],Sten[0,2]]]).T
    
    C_el=np.zeros([6,6])
    index1=[[0,0],[1,1],[2,2],[0,1],[1,2],[0,2]]
    index2=[[1,0],[2,1],[2,0]]
    for i in np.arange(C_el.shape[0]):
        for j in np.arange(i,C_el.shape[1]):
            a,b=index1[i]
            c,d=index1[j]
            c1=lam*invC[a,b]*invC[c,d]+2.*(mu-lam*np.log(J))*invC[a,c]*invC[b,d]
            if j>=3:
                e,f=index2[j-3]
                c2=lam*invC[a,b]*invC[e,f]+2.*(mu-lam*np.log(J))*invC[a,e]*invC[b,f]
                C_el[i,j]=(c1+c2)/2.
            else:
                C_el[i,j]=c1
            if i!=j:
                C_el[j,i]=C_el[i,j]
    
    
    C_el=np.asmatrix(C_el)
    
    return S,C_el

Imprimiremos un ejemplo de outputs para esta rutina

In [6]:
mu,lam=1.5,50.
material_parameters=mu,lam
E=np.random.random([6,1])
state=E
parameters=material_parameters
print('Tensor Lagrangeano de deformación en notación de Voigt:')
print(E)
S,C=material_NLElasticity_NH(state, parameters)
print('--------------------------')
print('Tensor PK2 en notación de Voigt')
print(S)
print('--------------------------')
print('Tensor elástico material C en notación de Voigt')
print(C)


Tensor Lagrangeano de deformación en notación de Voigt:
[[0.91968831]
 [0.1432619 ]
 [0.6827628 ]
 [0.99056701]
 [0.02650699]
 [0.06332594]]
--------------------------
Tensor PK2 en notación de Voigt
[[ 22.97389394]
 [ 48.87583566]
 [ 20.35618014]
 [-16.52592943]
 [ -0.08846684]
 [ -0.38968169]]
--------------------------
Tensor elástico material C en notación de Voigt
[[-9.08611243e+00  1.33446888e+01  1.01814031e+01  6.99251161e+00
  -3.36726392e-01  1.64883542e-01]
 [ 1.33446888e+01 -4.42252468e+01  2.24769531e+01  1.54269217e+01
   8.25836172e-02 -5.30108930e-01]
 [ 1.01814031e+01  2.24769531e+01 -7.00590002e+00 -7.84221774e+00
   3.28693213e-02  1.44783884e-01]
 [ 6.99251161e+00  1.54269217e+01 -7.84221774e+00 -2.20765761e+01
   4.18130679e-01  6.01879417e-02]
 [-3.36726392e-01  8.25836172e-02  3.28693213e-02  4.18130679e-01
  -2.00397338e+01  6.99048431e+00]
 [ 1.64883542e-01 -5.30108930e-01  1.44783884e-01  6.01879417e-02
   6.99048431e+00 -9.08294156e+00]]


### Test de consistencia

Ahora, verificaremos que nuestra función pase el test de consistencia de 4to orden

In [7]:
def MaterialConsistencyTest(func,state,params):
    e=np.random.random(state.shape)
    e=e/np.linalg.norm(e) 
    e=np.asmatrix(e)
    h=1e-3
    
    x0=state
    R,TR=func(x0, params)   
    
    x1=x0+2*h*e
    R1,TR1=func(x1, params)
    x2=x0+h*e
    R2,TR2=func(x2, params) 
    x3=x0-h*e
    R3,TR3=func(x3, params)       
    x4=x0-2*h*e
    R4,TR4=func(x4, params)         
    
    error=np.linalg.norm((-R1+8*R2-8*R3+R4)/12.-TR*e*h)
    return error

state=E
params=material_parameters
func=material_NLElasticity_NH

error=MaterialConsistencyTest(func,state,params)
print('Test de consistencia material: '+str(error))

Test de consistencia material: 2.801763217443831e-13


y vemos que el error es menor a $10^{-12}$, por lo que se verifica el test de consistencia.